## Subword Tokenization

- Github:https://github.com/keep-steady/NLP_for_korean


In [2]:
%%time
# NSMC 데이터 로드
import pandas as pd
f_train = pd.read_csv('data/nsmc.txt', sep='\t')
train_pair = [(row[1], row[2]) for _, row in f_train.iterrows() if type(row[1]) == str]  # nan 제거

#  문장 및 라벨 데이터 추출
train_data  = [pair[0] for pair in train_pair]
train_label = [pair[1] for pair in train_pair]
print('data loading done!')
print('문장: %s' %(train_data[:3]))
print('라벨: %s' %(train_label[:3]))

# subword 학습을 위해 문장만 따로 저장
with open('data/train_tokenizer.txt', 'w', encoding='utf-8') as f:
    for line in train_data:
        f.write(line+'\n')

# subword 학습을 위해 문장만 따로 저장
with open('data/train_tokenizer.txt', 'r', encoding='utf-8') as f:
    test_tokenizer = f.read().split('\n')
print(test_tokenizer[:3])

num_word_list = [len(sentence.split()) for sentence in test_tokenizer]
print('\n코퍼스 평균/총 단어 갯수 : %.1f / %d' % (sum(num_word_list)/len(num_word_list), sum(num_word_list)))

data loading done!
문장: ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다']
라벨: [0, 1, 0]
['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다']

코퍼스 평균/총 단어 갯수 : 7.6 / 1137736
CPU times: user 10.7 s, sys: 294 ms, total: 11 s
Wall time: 10.1 s


### BPE 학습


* input_file : 내 한국어 데이터 경로를 지정한다.
* vocab_size : BPE의 단어수를 얼마로 할 것인가 이다. 너무 적으면 한 글자 단위로 쪼개지는 경향이 있고, 너무 많으면 쓸데없는 단어들이 만들어진다. 주로 3,2000이 가장 좋다고 알려져 있다.
* model_name : 저장할 이름이다. 학습하고 나면 <model_name>.model, <model_name>.vocab 2개의 파일이 만들어진다.
* model_type : bpe, unigram 등이 있는데 두가지를 모두 사용해 보고 성능이 좋은 거로 고르도록 하자. 
* character_coverage : 모든 단어를 커버할것인가, 너무 희귀한 단어는 뺄 것인가 이다. 학습 코퍼스가 대용량이라면 보통 0.9995로 사용하면 된다. 그런데 코퍼스가 작다면 1.0으로 지정하자. 그럼 [UNK]가 없다.
* user_defined_symbols : BPE로 생성된 단어 외 알고리즘에서 사용할 특수문자들을 지정한다.

In [5]:
%%time
import sentencepiece as spm

# spm_train --input=data/train_tokenizer.txt  --model_prefix=sentencepiece/sp --vocab_size=32000 character_coverage=1.0 --model_type="unigram"

input_file = 'data/train_tokenizer.txt'
vocab_size = 32000

sp_model_root='sentencepiece'
if not os.path.isdir(sp_model_root):
    os.mkdir(sp_model_root)
sp_model_name = 'tokenizer_%d' % (vocab_size)
sp_model_path = os.path.join(sp_model_root, sp_model_name)
model_type = 'unigram'  # 학습할 모델 선택, unigram이 더 성능이 좋음'bpe'
character_coverage  = 1.0  # 전체를 cover 하기 위해, default=0.9995
user_defined_symbols = '[PAD],[UNK],[CLS],[SEP],[MASK],[BOS],[EOS],[UNK0],[UNK1],[UNK2],[UNK3],[UNK4],[UNK5],[UNK6],[UNK7],[UNK8],[UNK9],[unused0],[unused1],[unused2],[unused3],[unused4],[unused5],[unused6],[unused7],[unused8],[unused9],[unused10],[unused11],[unused12],[unused13],[unused14],[unused15],[unused16],[unused17],[unused18],[unused19],[unused20],[unused21],[unused22],[unused23],[unused24],[unused25],[unused26],[unused27],[unused28],[unused29],[unused30],[unused31],[unused32],[unused33],[unused34],[unused35],[unused36],[unused37],[unused38],[unused39],[unused40],[unused41],[unused42],[unused43],[unused44],[unused45],[unused46],[unused47],[unused48],[unused49],[unused50],[unused51],[unused52],[unused53],[unused54],[unused55],[unused56],[unused57],[unused58],[unused59],[unused60],[unused61],[unused62],[unused63],[unused64],[unused65],[unused66],[unused67],[unused68],[unused69],[unused70],[unused71],[unused72],[unused73],[unused74],[unused75],[unused76],[unused77],[unused78],[unused79],[unused80],[unused81],[unused82],[unused83],[unused84],[unused85],[unused86],[unused87],[unused88],[unused89],[unused90],[unused91],[unused92],[unused93],[unused94],[unused95],[unused96],[unused97],[unused98],[unused99]'

input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --user_defined_symbols=%s --model_type=%s --character_coverage=%s'
cmd = input_argument%(input_file, sp_model_path, vocab_size,user_defined_symbols, model_type, character_coverage)

spm.SentencePieceTrainer.Train(cmd)
print('train done')

train done
CPU times: user 58.6 s, sys: 515 ms, total: 59.2 s
Wall time: 11.5 s


In [15]:
## check
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(sp_model_path))

sentence = "나는 오늘 아침밥을 먹었다."
tokens = sp.encode_as_pieces(sentence)
ids = sp.encode_as_ids(sentence)

print(ids)
print(tokens)

tokens = sp.decode_pieces(tokens)
ids = sp.decode_ids(ids)

print(ids)
print(tokens)

[424, 657, 6107, 3260, 131, 8508, 140, 120]
['▁나는', '▁오늘', '▁아침', '밥', '을', '▁먹었', '다', '.']
나는 오늘 아침밥을 먹었다.
나는 오늘 아침밥을 먹었다.
